In [1]:
import json
import requests
import yaml
import pandas as pd

In [2]:
yaml_file_name = "../api_text.yml"

with open(yaml_file_name, 'r') as test_users_file:
    data_yml = yaml.full_load(test_users_file)

client_id = data_yml['graph_app']['client_id']
client_secret = data_yml['graph_app']['client_secret']
tenant_id = data_yml['graph_app']['tenant_id']


In [3]:
def get_access_token():
    #url = f"https://login.microsoftonline.com/{tenant_id}/oauth/v2.0/token"
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    #url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/authorize?client_id={client_id}&response_type=code&redirect_uri=https%3A%2F%2Flocalhost&response_mode=query&scope=offline_access%20Mail.ReadWrite%20Mail.send&state=12345644"

    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": "https://graph.microsoft.com/.default"
    }
    try:
        response = requests.post(url, data=payload)
    except:
        raise RuntimeError("Não foi possível retornar o request do post para pegar o access token")
    
    print(f"Resposta da microsoft a requisição do access token: {response}")
    response_data = response.json()
    #print(response.json())
    access_token = response_data['access_token']
    return access_token

In [4]:
def send_email():
    access_token = get_access_token()
    print(access_token)
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # url = "https://graph.microsoft.com/v1.0/users/me/sendMail".format(sender_mail)
    url = "https://graph.microsoft.com/v1.0/users/anakin.skywalker@fitenergia.com.br/sendMail"

    payload = {
        "message": {
            "subject": "Test mail sent using Python",
            "body": {
                "contentType": "Text",
                "content": "test mail generated using python on 03/10/2024"
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": "marceloramalho.info@gmail.com"
                    }
                }
            ],
            "ccRecipients": [
                {
                    "emailAddress": {
                        "address": "marcelo.ramalho@fitenergia.com.br"
                    }
                }
            ]
        },
        "saveToSentItems": "true"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    try:
        print(f"Resposta da microsoft ao post para mandar e-mail: {response.text}")
    except:
        raise RuntimeError(f"Erro: não printou.")

    if response.status_code == 200:
        graph_data = response.json()
        #print("Graph API call result: ")
        #print(json.dumps(graph_data, indent=2))
    else:
        print("Error: ")
        #print(response.get("error"))
        #print(response.get("error_description"))
        #print(response.get("correlation_id"))

In [5]:
def read_email(messages_json_path, email_address, n_of_massages):
    access_token = get_access_token()
    print("Pegou")
    print(email_address)
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    #url = f"https://graph.microsoft.com/v1.0/users/anakin.skywalker@fitenergia.com.br/messages/$top=2"
    url = f"https://graph.microsoft.com/v1.0/users/{email_address}/messages?$top= {n_of_massages}"
    #?$select=sender,subject
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        graph_data = response.json()
        #print("Graph API call result: ")
        #print(json.dumps(graph_data, indent=2))
        messages = response.json().get('value', [])
        try:
            with open(messages_json_path, 'w+', encoding='utf-8') as file:
                json.dump(messages, file, ensure_ascii=False, indent=4)
                print("Mailbox messages saved in messages.json successfully")
        except:
            raise RuntimeError("Não foi possível salvar as mensagens no arquivo messages.json")
        #    print("Subject:", message.get('subject'))
        #    print("From:", message.get('from', {}).get('emailAddress', {}).get('address'))
        #    print("Body Preview:", message.get('bodyPreview'))
        #    print("-------------------------------------------\n\n")
    else:
        print("Error:")
        print(response)
        #print(response.get("error"))
        #print(response.get("error_description"))
        #print(response.get("correlation_id"))

In [6]:
def json_to_dataframe(json_file_path):
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)
        print(f"Json_data: {json_data}")
    df_messages = pd.json_normalize(json_data)
    print("Json converted to dataframe successfully")
    return df_messages

def organize_df_messages(df_messages):
    #list(df_messages)
    columns = ['subject', 'isRead', 'sentDateTime', 'receivedDateTime', 'sender.emailAddress.name', 'sender.emailAddress.address', 'from.emailAddress.name', 'from.emailAddress.address', 'bodyPreview', 'body.contentType',
    'body.content']
    df_emails = df_messages[columns]
    print("DF clean created successfully")
    return df_emails

def get_df_messages(email_address, n_of_massages=10, messages_json_path='messages.json'):
    read_email(messages_json_path, email_address, n_of_massages)
    df_full_messages = json_to_dataframe(messages_json_path)
    df_messages = organize_df_messages(df_full_messages)
    return df_messages

In [ ]:
messages_json_path = 'messages.json'
#send_email()
#read_email()
#email_address = 'teste@fitenergia.com.br'
get_df_messages('teste@fitenergia.com.br')

In [8]:
#token = get_access_token()

In [9]:
#send_email()